<a href="https://colab.research.google.com/github/VitorThome10/FinanceDataAnalytics/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium
!pip install webdriver-manager  # gerencia os drivers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00


In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime

In [5]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
#driver = webdriver.Chrome(), options = options
# Isso evita o
driver.get("https://www.google.com/")

WebDriverException: ignored

In [ ]:
data_di = "27/07/2023"
mercadoria = 'WIN'


In [ ]:
url = f'''
https://www2.bmf.com.br/pages/portal/bmfbovespa/boletim1/SistemaPregao1.asp?pagetype=pop&caminho=Resumo%20
Estat%EDstico%20-%20Sistema%20Preg%E3o&Data={data_di}&Mercadoria={mercadoria} '''



In [ ]:
driver.get(url)


In [ ]:
driver.implicitly_wait(3)  # espera o códico abrir

driver.maximize_window()


local_table = '''/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[3]/table'''
indice_table = '''/html/body/div/div[2]/form[1]/table[3]/tbody/tr[3]/td[1]/table'''

elemento = driver.find_element("xpath", local_table)

elemento_indice = driver.find_element("xpath",indice_table)

html_table = elemento.get_attribute('outerHTML')
html_indice = elemento_indice.get_attribute('outerHTML')

tabela = pd.read_html(html_table)[0]
indice = pd.read_html(html_indice)[0]


In [ ]:
tabela.columns = tabela.loc[0]

tabela = tabela['ÚLT. PREÇO']

tabela = tabela.drop(0, axis = 0)

indice.columns = indice.loc[0]

indice_di = indice['VENCTO']

indice = indice.drop(0, axis = 0)

tabela.index = indice['VENCTO']

tabela = tabela.astype(float)

tabela = tabela[tabela != 0]


print(tabela)

In [ ]:
# Trata datas

legenda = pd.Series(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct',
                     'Nov','Dec'],
                   index = ['F','G','H','J','K','M','N','Q','U','V','X','Z'])

lista_datas = []

for indice in tabela.index:
    letra = indice[0]
    ano = indice[1:3]

    mes = legenda[letra]

    data = f'{mes}-{ano}'

    data = datetime.strptime(data, "%b-%y")

    lista_datas.append(data)

tabela.index = lista_datas

tabela